## Setup

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !ls /content/drive/MyDrive/ColabNotebooks/Transformer


In [3]:
# !nvcc --version

In [4]:
# !pip3 install torch==1.10.2+cu113 torchvision==0.11.3+cu113 torchaudio==0.10.2+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

In [5]:
# !pip3 install timm faiss tqdm numpy

In [6]:
# !sudo apt-get install libomp-dev

In [1]:
import torch

print(f'torch.__version__             = {torch.__version__}')
print(f'torch.cuda.is_available()     = {torch.cuda.is_available()}')
print(f'torch.cuda.current_device()   = {torch.cuda.current_device()}')
print(f'torch.cuda.device(0)          = {torch.cuda.device(0)}')
print(f'torch.cuda.device_count()     = {torch.cuda.device_count()}')
print(f'torch.cuda.get_device_name(0) = {torch.cuda.get_device_name(0)}')

torch.__version__             = 1.10.2+cu113
torch.cuda.is_available()     = True
torch.cuda.current_device()   = 0
torch.cuda.device(0)          = <torch.cuda.device object at 0x0000021D1876AF48>
torch.cuda.device_count()     = 1
torch.cuda.get_device_name(0) = NVIDIA GeForce GTX 1080


In [2]:
#!git clone https://github.com/SiddhantKapil/LA-Transformer.git
#!mv /content/LA-Transformer /content/drive/MyDrive/ColabNotebooks/Transformer

In [3]:
#%cd /content/drive/MyDrive/ColabNotebooks/Transformer/LA-Transformer

In [4]:
# Om inte detta fungerar, kopiera filen net_best.pth från författarnas Google Drive till eran egen, mount:a eran Google Drive, och kopiera filen till /content/LA-Transformer/net_best.pth
# %%shell
# if [ ! -f net_best.pth ]
# then
#     gdown --id 1qlvPquVLfP20neHYZ6cwzmzAw9s-O0Yf -O net_best.pth
# fi

In [5]:
# !unzip net_best.zip

In [6]:
# Om inte detta fungerar, kopiera filen Market_pytorch.zip från författarnas Google Drive till eran egen, mount:a eran Google Drive, och kopiera filen till /content/LA-Transformer/Market_pytorch.zip
# %%shell
# if [ ! -f Market_pytorch.zip ]
# then
#     unzip Market_pytorch.zip -d data
# fi

In [7]:
# !unzip Market_pytorch.zip

In [8]:
# %%shell
# if [ ! -d data ] && [ -f Market_pytorch.zip ]
# then
#     unzip Market_pytorch.zip -d data
# fi

# Load own data

In [9]:
# !unzip "/content/drive/MyDrive/ColabNotebooks/cleaned_reid.zip" -d data

# !unrar x "/content/drive/MyDrive/ColabNotebooks/NewSet_traintestval.rar" -d data

# Training

## Import Libraries

In [10]:
from __future__ import print_function

import os
import time
import random
import zipfile
from itertools import chain

import timm
import numpy as np
from PIL import Image
from tqdm.notebook import tqdm
from collections import OrderedDict

import torch
import torch.nn as nn
from torch.nn import init
import torch.optim as optim
from torchvision import models
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset

from LATransformer.model import ClassBlock, LATransformer
from LATransformer.utils import save_network, update_summary

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

# os.environ['CUDA_VISIBLE_DEVICES']='1'
device = "cuda"

## Set Config Parameters

In [11]:
batch_size = 16
num_epochs = 32
lr = 3e-4     # 0.0003
gamma = 0.7
unfreeze_after=2
lr_decay=.8
lmbd = 8

## Load Data

In [12]:
transform_train_list = [
    transforms.Resize((224,224), interpolation=3),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]
transform_val_list = [
    transforms.Resize(size=(224,224),interpolation=3), #Image.BICUBIC
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]
data_transforms = {
'train': transforms.Compose( transform_train_list ),
'val': transforms.Compose(transform_val_list),
}

E:\Anaconda\envs\py37\lib\site-packages\torchvision\transforms\transforms.py:288: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


In [13]:
image_datasets = {}
# data_dir = "data/Market-Pytorch/Market/"
data_dir = "data/mega_hyper_3000_resize/"

image_datasets['train'] = datasets.ImageFolder(os.path.join(data_dir, 'train'),
                                          data_transforms['train'])
image_datasets['val'] = datasets.ImageFolder(os.path.join(data_dir, 'val'),
                                          data_transforms['val'])
train_loader = DataLoader(dataset = image_datasets['train'], batch_size=batch_size, shuffle=True, num_workers=2)
valid_loader = DataLoader(dataset = image_datasets['val'], batch_size=batch_size, shuffle=True, num_workers=2)
# dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=opt.batchsize,
#                                              shuffle=True, num_workers=8, pin_memory=True) # 8 workers may work faster
#               for x in ['train', 'val']}
# dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
print(len(class_names))

50


## Load Model

In [14]:
# # Load pre-trained ViT
# vit_base = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=50)
# vit_base= vit_base.to(device)
# vit_base.eval()

## Train

In [15]:
class AverageMeter:
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [16]:
def validate(model, loader, loss_fn):
    batch_time_m = AverageMeter()
    losses_m = AverageMeter()
    top1_m = AverageMeter()
    top5_m = AverageMeter()

    model.eval()
    epoch_accuracy = 0
    epoch_loss = 0
    end = time.time()
    last_idx = len(loader) - 1
    
    running_loss = 0.0
    running_corrects = 0.0

    with torch.no_grad():
        for input, target in tqdm(loader):

            input, target = input.to(device), target.to(device)
            
            output = model(input)
            
            score = 0.0
            sm = nn.Softmax(dim=1)
            for k, v in output.items():
                score += sm(output[k])
            _, preds = torch.max(score.data, 1)

            loss = 0.0
            for k,v in output.items():
                loss += loss_fn(output[k], target)


            batch_time_m.update(time.time() - end)
            acc = (preds == target.data).float().mean()
            epoch_loss += loss/len(loader)
            epoch_accuracy += acc / len(loader)
            
            print(f"Epoch : {epoch+1} - val_loss : {epoch_loss:.4f} - val_acc: {epoch_accuracy:.4f}", end="\r")
    print()    
    metrics = OrderedDict([('val_loss', epoch_loss.data.item()), ("val_accuracy", epoch_accuracy.data.item())])


    return metrics

In [17]:
def train_one_epoch(
        epoch, model, loader, optimizer, loss_fn, unfrozen_blocks, 
        lr_scheduler=None, saver=None, output_dir='', 
        loss_scaler=None, model_ema=None, mixup_fn=None):

 

    
    batch_time_m = AverageMeter()
    data_time_m = AverageMeter()
    losses_m = AverageMeter()

    model.train()
    epoch_accuracy = 0
    epoch_loss = 0
    end = time.time()
    last_idx = len(loader) - 1
    num_updates = epoch * len(loader)
    running_loss = 0.0
    running_corrects = 0.0

    for data, target in tqdm(loader):
        data, target = data.to(device), target.to(device)

            
        data_time_m.update(time.time() - end)

        optimizer.zero_grad()
        output = model(data)
        score = 0.0
        sm = nn.Softmax(dim=1)
        for k, v in output.items():
            score += sm(output[k])
        _, preds = torch.max(score.data, 1)
        
        loss = 0.0
        for k,v in output.items():
            loss += loss_fn(output[k], target)
        loss.backward()

        optimizer.step()

        batch_time_m.update(time.time() - end)
        
#         print(preds, target.data)
        acc = (preds == target.data).float().mean()
        
#         print(acc)
        epoch_loss += loss/len(loader)
        epoch_accuracy += acc / len(loader)
#         if acc:
#             print(acc, epreds, target.data)
        print(
    f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f}", end="\r")
    writer.add_scalar("Loss/train",epoch_loss, epoch+1)

    writer.flush()

    name = "oprey_{}".format(lmbd)

    output_dir = "model/" + name

    path = os.path.join(output_dir, "saves", "model_{}.pt").format(epoch+1)

    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': epoch_loss,
        'accuracy': epoch_accuracy, 
        'unfrozen_blocks': unfrozen_blocks, 
        }, path)

    print()

    return OrderedDict([('train_loss', epoch_loss.data.item()), ("train_accuracy", epoch_accuracy.data.item())])

In [18]:
def freeze_all_blocks(model):
    frozen_blocks = 12
    for block in model.model.blocks[:frozen_blocks]:
        for param in block.parameters():
            param.requires_grad=False

In [19]:
def unfreeze_blocks(model, amount= 1):
    
    for block in model.model.blocks[11-amount:]:
        # print("block: ", block)
        for param in block.parameters():
            param.requires_grad=True
            # print("len param:", len(param))
    return model

## Training Loop

In [20]:
# # Load pre-trained ViT
# vit_base = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=50)
# vit_base= vit_base.to(device)
# # vit_base.eval()

# # Create LA Transformer
# model = LATransformer(vit_base, lmbd).to(device)
# # print(model.eval())

# # loss function
# criterion = nn.CrossEntropyLoss()

# # optimizer
# optimizer = optim.Adam(model.parameters(),weight_decay=5e-4, lr=lr)

# # scheduler
# scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
# freeze_all_blocks(model)

# Re-load trained model

In [21]:
# # Load pre-trained ViT
# vit_base = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=50)
# vit_base= vit_base.to(device)
# vit_base.eval()

# # Create LA Transformer
# model = LATransformer(vit_base, lmbd).to(device)

# print(model.eval())

# # loss function
# criterion = nn.CrossEntropyLoss()

# # optimizer
# optimizer = optim.Adam(model.parameters(),weight_decay=5e-4, lr=lr)

# # scheduler not needed? 
# scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

# try reloading model 

# Load pre-trained ViT
vit_base = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=50)
vit_base= vit_base.to(device)
vit_base.eval()

# Create LA Transformer
model = LATransformer(vit_base, lmbd).to(device)
# print(model.eval())

# loss function
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(),weight_decay=5e-4, lr=lr)

name = "oprey_{}".format(lmbd)

output_dir = "model/" + name

path = os.path.join(output_dir, "saves", "model_31.pt")

checkpoint = torch.load(path)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch_number = checkpoint['epoch'] + 1
loss = checkpoint['loss']
accuracy = checkpoint['accuracy']
unfrozen_blocks = checkpoint['unfrozen_blocks']
# unfrozen_blocks = 1

freeze_all_blocks(model)

print(f"Unfrozen  {unfrozen_blocks} blocks ")

first = True

optimizer
# unfrozen_blocks


#TODO skapa mindre dataset och testa på så allting fungerar kanske ?


Unfrozen  16 blocks 


Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    initial_lr: 0.0003
    lr: 6.7553994410557506e-06
    weight_decay: 0.0005
)

In [22]:
#
# OBS! Jag avbröt denna cellen under den 4:e epoken eftersom jag bara vill testa att koden fungerade (annars tränar den i 30 epoker)
#

from pathlib import Path

best_acc = 0.0
y_loss = {} # loss history
y_loss['train'] = []
y_loss['val'] = []
y_err = {}
y_err['train'] = []
y_err['val'] = []
print("training...")
output_dir = ""
best_acc = 0
# name = "la_with_lmbd_{}".format(lmbd)
name = "oprey_{}".format(lmbd)


try:
    Path("model/" + name).mkdir(parents=True, exist_ok=True)
except:
    pass
output_dir = "model/" + name


# unfrozen_blocks = 0

for epoch in range(num_epochs):


    epoch = epoch + epoch_number
    if epoch >= num_epochs:
        break

    print(f"Prevously: Epoch: {epoch}, Unfrozen blocks: {unfrozen_blocks}")

    print(epoch%unfreeze_after)

    if first == True:
        for i in range(unfrozen_blocks):
            model = unfreeze_blocks(model, i)
        print("first")

    if epoch%unfreeze_after==0:
        
        unfrozen_blocks += 1
        model = unfreeze_blocks(model, unfrozen_blocks)
        optimizer.param_groups[0]['lr'] *= lr_decay 
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        print("Unfrozen Blocks: {}, Current lr: {}, Trainable Params: {}".format(unfrozen_blocks, 
                                                                             optimizer.param_groups[0]['lr'], 
                                                                             trainable_params))


    train_metrics = train_one_epoch(
        epoch, model, train_loader, optimizer, criterion, unfrozen_blocks,
        lr_scheduler=None, saver=None)


    eval_metrics = validate(model, valid_loader, criterion)



    # update summary
    update_summary(epoch, train_metrics, eval_metrics, os.path.join(output_dir, 'summary.csv'), write_header=True)

    # deep copy the model
    last_model_wts = model.state_dict()
    if eval_metrics['val_accuracy'] > best_acc:
        best_acc = eval_metrics['val_accuracy']
        save_network(model, epoch, name)
        print("SAVED!")
    
    first = False


training...
Prevously: Epoch: 31, Unfrozen blocks: 16
1
first


  0%|          | 0/12009 [00:00<?, ?it/s]

Epoch : 32 - loss : 0.0736 - acc: 0.9998


  0%|          | 0/5146 [00:00<?, ?it/s]

Epoch : 32 - val_loss : 0.1400 - val_acc: 0.9979
SAVED!


In [ ]:
# try reloading model 

# Load pre-trained ViT
# vit_base = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=50)
# vit_base= vit_base.to(device)
# vit_base.eval()

# # Create LA Transformer
# model = LATransformer(vit_base, lmbd).to(device)
# # print(model.eval())

# # loss function
# criterion = nn.CrossEntropyLoss()

# name = "oprey_{}".format(lmbd)

# output_dir = "model/" + name

# path = os.path.join(output_dir, "saves_test", "model.pt")

# checkpoint = torch.load(path)
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
# loss = checkpoint['loss']
# accuracy = checkpoint['accuracy']

# optimizer
# epoch
# loss
# accuracy


# Adam (
# Parameter Group 0
#     amsgrad: False
#     betas: (0.9, 0.999)
#     eps: 1e-08
#     initial_lr: 0.0003
#     lr: 0.0003
#     weight_decay: 0.0005
# )

# batch_size = 16
# num_epochs = 30
# lr = 3e-4     # 0.0001
# gamma = 0.7
# unfreeze_after=2
# lr_decay=.8
# lmbd = 8

0

# Testing

In [ ]:
# OBS! Ni vill endast köra denna cellen om ni vill testa författarnas bästa modell "net_best.pth" (tränad i 30 epoker) istället för eran egen modell som ni tränade ovan!
# Annars, skippa denna cellen!

# %%shell
# mkdir -p /content/LA-Transformer/model/la_with_lmbd_8
# cp net_best.pth /content/LA-Transformer/model/la_with_lmbd_8/net_best.pth

In [ ]:
def import_libraries():
    from __future__ import print_function

    import os
    import time
    import glob
    import random
    import zipfile
    from itertools import chain

    import timm
    import numpy as np
    import pandas as pd
    from PIL import Image
    from tqdm.notebook import tqdm
    import matplotlib.pyplot as plt
    from collections import OrderedDict
    from sklearn.model_selection import train_test_split

    import torch
    import torch.nn as nn
    from torch.nn import init
    import torch.optim as optim
    from torchvision import models
    import torch.nn.functional as F
    from torch.autograd import Variable
    from torch.optim.lr_scheduler import StepLR
    from torchvision import datasets, transforms
    from torch.utils.data import DataLoader, Dataset

    from LATransformer.model import ClassBlock, LATransformer, LATransformerTest
    from LATransformer.utils import save_network, update_summary, get_id
    from LATransformer.metrics import rank1, rank5, rank10, calc_map

# os.environ['CUDA_VISIBLE_DEVICES']='1'
device = "cuda"

# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") ## Use if CPU

# pattes kod 
#!pip install torch==1.10.2+cu113 torchvision==0.11.3+cu113 torchaudio==0.10.2+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
# Min cuda version
#!pip install torch==1.10.2+cu116 torchvision==0.11.3+cu116 torchaudio==0.10.2+cu116 -f https://download.pytorch.org/whl/cu116/torch_stable.html
# för cpu only
#conda install pytorch=1.10.2 torchvision=0.11.3 torchaudio=0.10.2 cpuonly -c pytorch


#!pip install timm faiss-cpu tqdm numpy pandas matplotlib scikit-learn
# conda install -c conda-forge ipywidgets

# import torch

if not device.type == "cpu":
    print(f'torch.__version__             = {torch.__version__}')
    print(f'torch.cuda.is_available()     = {torch.cuda.is_available()}')
    print(f'torch.cuda.current_device()   = {torch.cuda.current_device()}')
    print(f'torch.cuda.device(0)          = {torch.cuda.device(0)}')
    print(f'torch.cuda.device_count()     = {torch.cuda.device_count()}')
    print(f'torch.cuda.get_device_name(0) = {torch.cuda.get_device_name(0)}')


True


In [ ]:
# import torch

# print(f'torch.__version__             = {torch.__version__}')
# print(f'torch.cuda.is_available()     = {torch.cuda.is_available()}')
# print(f'torch.cuda.current_device()   = {torch.cuda.current_device()}')
# print(f'torch.cuda.device(0)          = {torch.cuda.device(0)}')
# print(f'torch.cuda.device_count()     = {torch.cuda.device_count()}')
# print(f'torch.cuda.get_device_name(0) = {torch.cuda.get_device_name(0)}')

## Config Parameters

In [ ]:
# batch_size = 8
# gamma = 0.7
# seed = 42

## Load Model

In [ ]:
batch_size = 8
gamma = 0.7
seed = 42

# Load ViT
vit_base = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=14)
vit_base= vit_base.to(device)

# Create La-Transformer
model = LATransformerTest(vit_base, lmbd=8).to(device)


# checkpoint = torch.load(path)
# model.load_state_dict(checkpoint['model_state_dict'])

name = "oprey_{}".format(lmbd)
output_dir = "model/" + name
path = os.path.join(output_dir, "saves", "model_30.pt")

# Load LA-Transformer
# name = "la_with_lmbd_8"
# name = "la_with_lmbd_8_12-03"
# name = "oprey"
# save_path = os.path.join('./model',name,'net_best.pth')

checkpoint = torch.load(path)
model.load_state_dict(checkpoint['model_state_dict'])



# # Load LA-Transformer
# name = "old_weights"
# save_path = os.path.join('./model',name,'small_ds_68_map_net_best.pth')

#Transformer\model\old_weights\small_ds_68_map_net_best.pth



# model.load_state_dict(torch.load(save_path), strict=False)
# model.eval()

transform_query_list = [
    transforms.Resize((224,224), interpolation=3),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]
transform_gallery_list = [
    transforms.Resize(size=(224,224),interpolation=3), #Image.BICUBIC
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]
data_transforms = {
'query': transforms.Compose( transform_query_list ),
'gallery': transforms.Compose(transform_gallery_list),
}

C:\Users\jimhu\anaconda3\envs\py37\lib\site-packages\torchvision\transforms\transforms.py:288: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


## DataLoader

In [ ]:
# transform_query_list = [
#     transforms.Resize((224,224), interpolation=3),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]
# transform_gallery_list = [
#     transforms.Resize(size=(224,224),interpolation=3), #Image.BICUBIC
#     transforms.ToTensor(),
#     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]
# data_transforms = {
# 'query': transforms.Compose( transform_query_list ),
# 'gallery': transforms.Compose(transform_gallery_list),
# }

C:\Users\jimhu\anaconda3\envs\py37\lib\site-packages\torchvision\transforms\transforms.py:288: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


In [ ]:
# Unzip test data

#!unzip "/content/drive/MyDrive/ColabNotebooks/datasets/test_folder.zip"

In [ ]:
# do_the_shit(data_dir_path)
#     image_datasets = {}
#     # data_dir = "data/Market-Pytorch/Market/"
#     # data_dir = "test_folder"
#     # data_dir = "data/The_OspreyChallengerSet"
#     data_dir = data_dir_path

IndentationError: unexpected indent (1991116892.py, line 2)

In [ ]:
# We had to recreate the get_id() func since they assume the pictures are named in a specific manner. 
def get_id_padel(img_path):

  # camera_id = []
  labels = []

  for path, v in img_path:
      #filename = path.split('/')[-1]
      filename = os.path.basename(path)
      # label = filename[0:4]

      label = filename.split('_')[0]

      labels.append(int(label))
      # if label[0:2]=='-1':
      #     labels.append(-1)
      # else:
      #     labels.append(int(label))
      # camera_id.append(int(camera[0]))


  return labels

In [ ]:
image_datasets = {}
# data_dir = "data/Market-Pytorch/Market/"
# data_dir = "test_folder"
data_dir = "data/The_OspreyChallengerSet"

image_datasets['query'] = datasets.ImageFolder(os.path.join(data_dir, 'query'),
                                          data_transforms['query'])
image_datasets['gallery'] = datasets.ImageFolder(os.path.join(data_dir, 'gallery'),
                                          data_transforms['gallery'])
query_loader = DataLoader(dataset = image_datasets['query'], batch_size=batch_size, shuffle=False )
gallery_loader = DataLoader(dataset = image_datasets['gallery'], batch_size=batch_size, shuffle=False)

class_names = image_datasets['query'].classes
print(len(class_names))

activation = {}

def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

def extract_feature(model,dataloaders):
    
    features =  torch.FloatTensor()
    count = 0
    idx = 0
    for data in tqdm(dataloaders):
        img, label = data
        img, label = img.to(device), label.to(device)

        output = model(img)

        n, c, h, w = img.size()
        
        count += n
        features = torch.cat((features, output.detach().cpu()), 0)
        idx += 1
    return features

# Extract Query Features
query_feature = extract_feature(model, query_loader)

# Extract Gallery Features
gallery_feature = extract_feature(model, gallery_loader)



#Retrieve labels
gallery_path = image_datasets['gallery'].imgs
query_path = image_datasets['query'].imgs

# print("gallery_path: ", gallery_path)
# print("query path: ", query_path)

gallery_label = get_id_padel(gallery_path)
query_label = get_id_padel(query_path)

concatenated_query_vectors = []
for query in tqdm(query_feature):
   
    fnorm = torch.norm(query, p=2, dim=1, keepdim=True)*np.sqrt(14)
   
    query_norm = query.div(fnorm.expand_as(query))
    
    concatenated_query_vectors.append(query_norm.view((-1))) # 14*768 -> 10752

concatenated_gallery_vectors = []
for gallery in tqdm(gallery_feature):
   
    fnorm = torch.norm(gallery, p=2, dim=1, keepdim=True) *np.sqrt(14)
   
    gallery_norm = gallery.div(fnorm.expand_as(gallery))
    
    concatenated_gallery_vectors.append(gallery_norm.view((-1))) # 14*768 -> 10752

import faiss
import numpy as np


index = faiss.IndexIDMap(faiss.IndexFlatIP(10752))

index.add_with_ids(np.array([t.numpy() for t in concatenated_gallery_vectors]), np.array(gallery_label).astype('int64'))  # original 


def search(query: str, k=1):
    encoded_query = query.unsqueeze(dim=0).numpy()
    top_k = index.search(encoded_query, k)
    return top_k

IndentationError: unexpected indent (458696210.py, line 2)

In [ ]:
for query, label in zip(concatenated_query_vectors, query_label):
    output = search(query)
    print("out: ", output)

out:  (array([[0.93609476]], dtype=float32), array([[4]], dtype=int64))


## Extract Features

In [ ]:
# activation = {}
# def get_activation(name):
#     def hook(model, input, output):
#         activation[name] = output.detach()
#     return hook

In [ ]:
# def extract_feature(model,dataloaders):
    
#     features =  torch.FloatTensor()
#     count = 0
#     idx = 0
#     for data in tqdm(dataloaders):
#         img, label = data
#         img, label = img.to(device), label.to(device)

#         output = model(img)

#         n, c, h, w = img.size()
        
#         count += n
#         features = torch.cat((features, output.detach().cpu()), 0)
#         idx += 1
#     return features

In [ ]:
# # Extract Query Features
# query_feature = extract_feature(model, query_loader)

# # Extract Gallery Features
# gallery_feature = extract_feature(model, gallery_loader)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# # We had to recreate the get_id() func since they assume the pictures are named in a specific manner. 
# def get_id_padel(img_path):

#   # camera_id = []
#   labels = []

#   for path, v in img_path:
#       #filename = path.split('/')[-1]
#       filename = os.path.basename(path)
#       # label = filename[0:4]

#       label = filename.split('_')[0]

#       labels.append(int(label))
#       # if label[0:2]=='-1':
#       #     labels.append(-1)
#       # else:
#       #     labels.append(int(label))
#       # camera_id.append(int(camera[0]))


#   return labels

In [ ]:
# #Retrieve labels
# gallery_path = image_datasets['gallery'].imgs
# query_path = image_datasets['query'].imgs

# # print("gallery_path: ", gallery_path)
# # print("query path: ", query_path)

# gallery_label = get_id_padel(gallery_path)
# query_label = get_id_padel(query_path)

# print("galley labels: ", gallery_label)

# gallery_cam, gallery_label = get_id(gallery_path)
# query_cam, query_label = get_id(query_path)

## Concat Averaged GELTs

In [ ]:
# concatenated_query_vectors = []
# for query in tqdm(query_feature):
   
#     fnorm = torch.norm(query, p=2, dim=1, keepdim=True)*np.sqrt(14)
   
#     query_norm = query.div(fnorm.expand_as(query))
    
#     concatenated_query_vectors.append(query_norm.view((-1))) # 14*768 -> 10752

# concatenated_gallery_vectors = []
# for gallery in tqdm(gallery_feature):
   
#     fnorm = torch.norm(gallery, p=2, dim=1, keepdim=True) *np.sqrt(14)
   
#     gallery_norm = gallery.div(fnorm.expand_as(gallery))
    
#     concatenated_gallery_vectors.append(gallery_norm.view((-1))) # 14*768 -> 10752

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

In [ ]:

# for i, v in enumerate(gallery_label):
#   print(f"{concatenated_gallery_vectors[i]}, {v}")

## Calculate Similarity using FAISS

In [ ]:
# import faiss
# import numpy as np


# index = faiss.IndexIDMap(faiss.IndexFlatIP(10752))

# # Our own attempt to fix
# # -------- Start
# # 14*768 -> 10752 from the cell above 

# # data = np.array([t.numpy() for t in concatenated_gallery_vectors])

# # print(len(data), len(concatenated_gallery_vectors), len(gallery_label))
# # # print(yes.shape, concatenated_gallery_vectors.shape, gallery_label.shape)

# # print(data.shape)
# # print(np.array(gallery_label).shape)


# # index.add_with_ids(data, np.array(gallery_label))

# # index.add_with_ids(data, np.array(gallery_label).astype(np.int64)) # Runs but does not work


# # Problem might be that the "np.array(gallery_label)" has shape (2000, ) and should be matching "data" shape which is (2000, 10752)
# # -------- End

# # Uncomment cell bellow to run original code
# index.add_with_ids(np.array([t.numpy() for t in concatenated_gallery_vectors]), np.array(gallery_label).astype('int64'))  # original 



# # xb = np.array([t.numpy() for t in concatenated_gallery_vectors]).astype(dtype=np.float32)
# # index = faiss.IndexFlatL2(10752) 
# # ids = np.array(gallery_label, dtype=np.float32)
# # index2 = faiss.IndexIDMap(index)
# # index2.add_with_ids(xb, ids)


# def search(query: str, k=1):
#     encoded_query = query.unsqueeze(dim=0).numpy()
#     top_k = index.search(encoded_query, k)
#     return top_k

In [ ]:
# for query, label in zip(concatenated_query_vectors, query_label):
#     output = search(query)
#     print("out: ", output)


out:  (array([[0.81973946]], dtype=float32), array([[3]], dtype=int64))


In [ ]:
rank1_score = 0
rank5_score = 0
rank10_score = 0
ap = 0
count = 0
for query, label in zip(concatenated_query_vectors, query_label):
    count += 1
    label = label
    output = search(query, k=10)
#     print(output)
    rank1_score += rank1(label, output) 
    rank5_score += rank5(label, output) 
    rank10_score += rank10(label, output) 
    print("Correct: {}, Total: {}, Incorrect: {}".format(rank1_score, count, count-rank1_score), end="\r")
    ap += calc_map(label, output)

print("Rank1: {}, Rank5: {}, Rank10: {}, mAP: {}".format(rank1_score/len(query_feature), 
                                                         rank5_score/len(query_feature), 
                                                         rank10_score/len(query_feature), ap/len(query_feature))) 

Rank1: 1.0, Rank5: 1.0, Rank10: 1.0, mAP: 1.0


In [ ]:
import_libraries()

do_the_shit("data/The_OspreyChallengerSet")